In [8]:
#k center problem with outliers
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.datasets import make_blobs

class KCenterProblemWithOutliers():
  def __init__(self, x_input, y_input) -> None:
    self.x_input, self.y_input = x_input, y_input
    self.NUM_INPUTS = len(x_input)
    self.dists = [[0 for _ in range(self.NUM_INPUTS)] for _ in range(self.NUM_INPUTS)]
    self.selected_samples = []
    self.center_dists = [float('inf') for _ in range(self.NUM_INPUTS)]

    self.compute_dists()

  def compute_dists(self):
    for r in range(self.NUM_INPUTS):
      for c in range(self.NUM_INPUTS):
        if not r == c:
          self.dists[r][c] = math.sqrt((self.x_input[r] - self.x_input[c])**2 + (self.y_input[r] - self.y_input[c])**2)

  def select_samples(self, query_size):
    # ******************************** Helper ********************************
    def maxindex(dist):
      max_ind = 0
      for i in range(self.NUM_INPUTS):
          if (dist[i] > dist[max_ind]):
              max_ind = i
      return max_ind

    # ******************************* Algortihm ******************************
    max_center = 0

    for _ in range(query_size):
      self.selected_samples.append(max_center)
      
      for j in range(self.NUM_INPUTS):
        self.center_dists[j] = min(self.center_dists[j], self.dists[max_center][j])
      
      max_center = maxindex(self.center_dists)
    # ************************************************************************

    x_query, y_query = [], []
    for sample_ind in self.selected_samples:
      x_query.append(self.x_input[sample_ind])
      y_query.append(self.y_input[sample_ind])

    self.plot_points(x_input, y_input, x_query, y_query)

    return self.selected_samples

  def plot_points(self, x_input, y_input, x_query, y_query):
    # plt.scatter(x=x_input, y=y_input, s=1, color='blue')
    plt.scatter(x=x_query, y=y_query, s=10, color='red')
    plt.show()

if __name__ == '__main__' :
  num_inputs = 15_000
  query_size = 15

  centers = [(0, -3), (1, 4), (3,2), (3,4), (1,1), (6,-1), (7,1)]
  cluster_std = [1,1,1,1,1,1,1]

  x, y = make_blobs(n_samples=num_inputs, cluster_std=cluster_std, centers=centers, n_features=2, random_state=1)

  x_input = x[:,0].tolist()
  y_input = x[:,1].tolist()

  clusters = [[] for _ in range(len(centers))]

  for ind, label in enumerate(y):
    clusters[label].append(x[ind])

  colors = ['blue', 'green', 'purple', 'black', 'pink', 'olive', 'gray']
  
  x_coordinates = [[] for _ in range(len(centers))]
  y_coordinates = [[] for _ in range(len(centers))]

  for ind, cluster in enumerate(clusters):
    for x, y in cluster:
        x_coordinates[ind].append(x)
        y_coordinates[ind].append(y)

    plt.scatter(x_coordinates[ind], y_coordinates[ind], color=colors[ind], s=1)
  
  for _ in range(1):
    if num_inputs // query_size > 0:
      query = KCenterProblemWithOutliers(x_input, y_input)

      prev_query_inds = query.select_samples(query_size)
      prev_query_inds.sort(reverse=True)

      for ind in prev_query_inds:
        x_input.pop(ind)
        y_input.pop(ind)

In [36]:
#K Center Problem for tensors

import torch

#helper function **********************************************************************
def max_ind(dist):
  max_ind = 0
  for i in range(num_inputs):
      if (dist[i] > dist[max_ind]):
          max_ind = i
  return max_ind

#algorithm *****************************************************************************
num_inputs = 100
dim = 10
query_size = 8
inputs = []

for _ in range(num_inputs):
  inputs.append(torch.rand(1,dim))
  
dists_between_inputs = torch.zeros(num_inputs, num_inputs)

for r in range(num_inputs):
  for c in range(num_inputs):
    dists_between_inputs[r][c] = torch.cdist(inputs[r], inputs[c])

dists_from_centers = torch.Tensor([float('inf') for _ in range(num_inputs)])

max_center_ind = 0
selected_samples_inds = []

for _ in range(query_size):
  selected_samples_inds.append(max_center_ind)

  for i in range(num_inputs):
    dists_from_centers[i] = min(dists_from_centers[i], dists_between_inputs[max_center_ind][i])

  max_center_ind = max_ind(dists_from_centers)

print(selected_samples_inds)

[0, 90, 24, 80, 44, 5, 94, 57]
